In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import calendar
import datetime as dt
from itertools import chain
from datetime import datetime, timedelta
import glob
import numpy as np
import pandas as pd

In [2]:
# -- STORE ALL FITBIT DATA INTO LISTS OF SHEET --
xls_files = glob.glob('Fitbit/*.xls')

## Functions

In [3]:
def add_weekday(df, not_index=0):
    """Add day of week column"""
    
    # -- USE DATAFRAME'S INDEX IF 'not_index' = 1, OTHERWISE USE DATAFRAME'S 'Date' COLUMN --
    col = df.index
    if not_index:
        col = df['Date']
    
    # -- DETERMINE THE DAY OF WEEK FROM THE DATE AND STORE IN A LIST (USED TO ADD AS A COLUMN TO THE DATAFRAME) --
    weekday = []
    for date in col:
        #weekday.append(datetime.strptime(str(date), '%Y-%m-%d').strftime('%A'))
        weekday.append(calendar.day_name[date.weekday()])
    df.loc[:, 'Weekday'] = weekday
    
    
def to_num(df, list_of_cols):
    """Remove commas and convert values to a numeric data type"""
    
    for col in list_of_cols:
        #df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('\D', '', regex=True)
        df[col] = pd.to_numeric(df[col])
        

def to_date(df, list_of_col):
    """Convert columns into datetime type"""
    
    for col in list_of_col:
        df[col] = pd.to_datetime(df[col])
        
        
def replace_outliers(df):
    """Replace outliers with mean"""
    
    for col in df.columns:
        mean = df[col].mean()
        std = df[col].std()
        df[col] = df[col].mask(((df[col] - mean).abs() > 2 * std), mean)

### Function for wrangling sleep data

In [4]:
def sort_sleep_data(list_of_df):
    """Sort sleep data by date"""
    
    for i in range(len(list_of_df)):
        list_of_df[i].sort_values(by='Start Time', ascending=True, inplace=True)



### Functions for wrangling food and macros data

In [5]:
def flatten_list(l):
    """Flatten a list"""
    
    return list(chain(*l))


def get_sheetnames_and_dates(excel_file):
    """Only get sheet names and dates for days where food data is entered"""
    """Returns: 2 lists (sheet names & dates)"""
    
    monthly_calories = pd.ExcelFile(excel_file).parse('Foods')

    dates = []
    sheet_names = []
    
    # -- GET SHEET NAMES AND DATES ONLY FOR THE DAYS WHERE INFO ABOUT FOOD INTAKE IS ENTERED --
        # -- daily[0]: date --
        # -- daily[1]: calories --
    for daily in monthly_calories.values:
        if str(daily[1]) != '0':
            dates.append(datetime.strptime(daily[0], '%Y-%m-%d').date())
            foodLogSheetName = 'Food Log ' + daily[0].replace('-', '')
            sheet_names.append(foodLogSheetName)
    return sheet_names, dates


def remove_empty_rows(xls, sheet_list):
    """Converting data in labeled table format to dataframe format"""
    """Returns: dataframe without unnecessary data"""
    
    df_list = [pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_list]

    # -- REMOVING UNNECESSARY ROWS & FILLING IN 'MISSING' DATA --
    for df in df_list:
        df.dropna(how='all', inplace=True)
        df['Meal'] = df['Meal'].fillna(method='ffill')
        df.reset_index(drop=True, inplace=True)
        
    return df_list


def get_food_and_macros(food_list, dates_list):
    """Get macros information into desirable dataset format"""
    """Returns: 2 dataframes (one with food consumption data & one with daily macros info)"""

    macros_df = pd.DataFrame()
    all_food_df = pd.DataFrame()
    all_macros_df = pd.DataFrame()
    
    # -- SPLITTING DATA INTO TWO DIFFERENT DATAFRAMES --
    for ind, (df, date) in enumerate(zip(food_list, dates_list)):
        
        index_of_macros = []

        # -- STORE INDEX OF ROWS THAT HAVE MACROS DATA --
        for i in df.index:
            if df['Meal'][i] == 'Daily Totals':
                index_of_macros.append(i)

        # -- EXTRACT MACROS DATA FROM THE DATAFRAME. CREATE & STORE MACROS DATA IN A NEW DATAFRAME --
        macros_df = df.iloc[index_of_macros[1]:index_of_macros[-1]+1]
        macros_df['Date'] = date
        macros_df = macros_df.pivot(index='Date', columns='Food', values='Calories')
        all_macros_df = all_macros_df.append(macros_df)

        # -- ADD 'Date' COLUMN TO THE FOOD DATAFRAME --
        food_list[ind]['Date'] = date

        # -- STORE EACH UPDATED FOOD DATAFRAME INTO A NEW DATAFRAME --
        all_food_df = all_food_df.append(food_list[ind].drop(index=index_of_macros).dropna())

    return all_food_df, all_macros_df



## Data wrangling

### Cleaning activities & sleep data

In [6]:
# -- STORING DATA IN EXCEL AS LISTS DATAFRAMES --
list_of_activities_df = [pd.ExcelFile(xls).parse('Activities', index_col=0, parse_dates=True) for xls in xls_files]
list_of_sleep_df = [pd.ExcelFile(xls).parse('Sleep', usecols=':F') for xls in xls_files]

# -- CONCATING ALL FOOD DATAFRAMES INTO ONE BIG DATAFRAME --
activities = pd.concat(list_of_activities_df)

# -- CONCATING ALL SLEEP DATAFRAMES INTO ONE BIG DATAFRAME --
sort_sleep_data(list_of_sleep_df)
sleep = pd.concat(list_of_sleep_df)
sleep.reset_index(drop=True, inplace=True)

# -- GET COLUMNS TO CORRECT TYPE & REMOVING MISSING VALUES --
to_num(activities, ['Calories Burned', 'Steps', 'Minutes Sedentary', 'Activity Calories'])
to_date(sleep, ['Start Time', 'End Time'])
activities = activities[activities['Steps'] != 0]

# -- CREATE A NEW SLEEP DATAFRAME WHERE THERE WILL ONLY BE ONE RECORD FOR EACH DAY --
dates = []
for i in range(len(sleep['Start Time'])):
    date = sleep['Start Time'][i].date()
    time = sleep['Start Time'][i].time()
    # -- BEDTIMES BEFORE 4AM ARE CONSIDERED TO BE PREVIOUS DAY'S SLEEP --
    if time > dt.time(4):
        dates.append(date)
    else:
        dates.append(date - timedelta(1))
sleep['Date'] = dates

daily_sleep = sleep.groupby('Date').sum()
daily_sleep.index = pd.to_datetime(daily_sleep.index)

# -- CREATE ANOTHER SLEEP DATAFRAME WITH NO NAP ENTRIES --
for x in sleep['Start Time']:
    x = x.to_pydatetime()

indices = []
for i, x in enumerate(sleep['Start Time']):
    if (x.time() > dt.time(20)) | (x.time() < dt.time(4)):
        indices.append(i)
        
sleep_night = sleep.loc[indices]
sleep_night = sleep_night.groupby('Date').sum()

# -- REPLACE OUTLIERS WITH MEAN --
replace_outliers(activities)
replace_outliers(daily_sleep)

# -- ADD DAY OF WEEK COLUMN --
add_weekday(activities)
add_weekday(daily_sleep)

### Cleaning food data

In [7]:
# -- GET ALL NECESSARY SHEET NAMES AND DATES --
sheet_names, dates = zip(*[get_sheetnames_and_dates(xls) for xls in xls_files])
dates = [d for d in dates if d]

# -- FIRST STEP IN DATA CLEANING: REMOVING ALL EMPTY ROWS --
list_of_food_df = [remove_empty_rows(xls, monthly_sheets) for xls, monthly_sheets in zip(xls_files, sheet_names) if monthly_sheets]

# -- SECOND STEP IN DATA CLEANING: SPLIT ORIGINAL DATAFRAMES INTO SEPARATE DATAFRAMES --
list_of_food_df, list_of_macros_df = zip(*[get_food_and_macros(monthly_food, date) for monthly_food, date in zip(list_of_food_df, dates)])

macros = pd.concat(list_of_macros_df)
food = pd.concat(list_of_food_df)
food.reset_index(drop=True, inplace=True)

to_num(macros, ['Calories', 'Carbs', 'Fat', 'Fiber', 'Protein', 'Sodium', 'Water'])

add_weekday(macros)
add_weekday(food, 1)

macros.columns = ['Calories (g)', 'Carbs (g)', 'Fat (g)', 'Fiber (g)', 'Protein (g)', 'Sodium (mg)', 'Water (fl oz)', 'Weekday']

In [8]:
activities.head()

,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,Weekday
Date,,,,,,,,,,
2015-10-21,2150.000000,14061.0,5.71,17.0,531.452381,324.060524,0.0,0.0,1588.812105,Wednesday
2015-10-22,2274.000000,13617.0,5.46,12.0,596.000000,300.000000,17.0,69.0,1344.000000,Thursday
2015-10-23,2174.000000,16530.0,6.57,20.0,639.000000,361.000000,15.0,35.0,1275.000000,Friday
2015-10-24,2161.000000,14710.0,5.88,11.0,550.000000,278.000000,36.0,52.0,1227.000000,Saturday
2015-10-25,2479.197832,5077.0,2.02,8.0,869.000000,324.060524,9.0,14.0,1588.812105,Sunday


In [9]:
sleep.head()

,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Date
0,2015-10-22 00:00:00,2015-10-22 05:07:00,292,15,1,307,2015-10-21
1,2015-10-22 21:29:00,2015-10-23 04:17:00,401,7,1,408,2015-10-22
2,2015-10-23 21:47:00,2015-10-24 06:43:00,514,22,2,536,2015-10-23
3,2015-10-24 23:24:00,2015-10-25 07:16:00,459,13,1,472,2015-10-24
4,2015-10-24 14:40:00,2015-10-24 16:05:00,80,5,0,85,2015-10-24


In [10]:
daily_sleep.head()

,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Weekday
Date,,,,,
2015-10-21,459.233849,15.0,1.0,496.392175,Wednesday
2015-10-22,401.000000,7.0,1.0,408.000000,Thursday
2015-10-23,514.000000,22.0,2.0,536.000000,Friday
2015-10-24,539.000000,18.0,1.0,557.000000,Saturday
2015-10-26,532.000000,33.0,2.0,565.000000,Monday


In [11]:
food.head()

,Meal,Food,Calories,Date,Weekday
0,Breakfast,American Cheese,61,2015-11-09,Monday
1,Breakfast,"Bagel thins, Everything",110,2015-11-09,Monday
2,Breakfast,"Egg, Chicken, Fried",184,2015-11-09,Monday
3,Breakfast,"Ham Steak, Traditional",30,2015-11-09,Monday
4,Morning Snack,Dark Chocolate Dreams,170,2015-11-09,Monday


In [12]:
macros.head()

,Calories (g),Carbs (g),Fat (g),Fiber (g),Protein (g),Sodium (mg),Water (fl oz),Weekday
Date,,,,,,,,
2015-11-09,715,72,34,8,35,943,0,Monday
2015-11-11,797,74,39,4,37,1064,0,Wednesday
2015-11-12,1049,108,45,11,53,1216,0,Thursday
2015-11-30,90,20,0,1,1,2,0,Monday
2015-12-02,240,29,6,3,17,152,0,Wednesday
